In [1]:
!PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0

In [4]:
!pip install auto_gptq==0.2.0
!pip install transformers==4.30
!pip install --upgrade optimum
!pip install --upgrade git+https://github.com/huggingface/transformers.git
!pip install --upgrade accelerate

  Obtaining dependency information for transformers==4.30 from https://files.pythonhosted.org/packages/e2/72/1af3d38e98fdcceb3876de4567ac395a66c26976e259fe2d46266e052d61/transformers-4.30.0-py3-none-any.whl.metadata
  Using cached transformers-4.30.0-py3-none-any.whl.metadata (113 kB)
  Obtaining dependency information for tokenizers!=0.11.3,<0.14,>=0.11.1 from https://files.pythonhosted.org/packages/70/68/0a450e4dc488031b82fcd869840c542b86aad4a07d0eca1d7e9cbb9d742e/tokenizers-0.13.3-cp310-cp310-macosx_12_0_arm64.whl.metadata
  Using cached tokenizers-0.13.3-cp310-cp310-macosx_12_0_arm64.whl.metadata (6.7 kB)
Using cached transformers-4.30.0-py3-none-any.whl (7.2 MB)
Using cached tokenizers-0.13.3-cp310-cp310-macosx_12_0_arm64.whl (3.9 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transform

In [7]:
from transformers import pipeline, Conversation, AutoTokenizer
from src.packages.utils.parameter_server import ParameterServer
import torch

parameter_server = ParameterServer()

base_model_name = "bardsai/jaskier-7b-dpo-v6.1"
generator = pipeline("conversational", model=base_model_name, torch_dtype=torch.half, device_map=parameter_server.settings.device, do_sample=False, top_k=0.95)
model = generator.model
model

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
tokenizer

LlamaTokenizerFast(name_or_path='bardsai/jaskier-7b-dpo-v6.1', vocab_size=32000, model_max_length=32768, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [17]:
from transformers import StoppingCriteriaList, MaxLengthCriteria, MinLengthLogitsProcessor, LogitsProcessorList

tokenizer.pad_token_id = tokenizer.eos_token_id
model.generation_config.pad_token_id = model.generation_config.eos_token_id
input_ids = tokenizer("Назви найбільші міста України та коротко опиши їх. Розкажи детальніше про Львів", return_tensors="pt").input_ids.to(device=parameter_server.settings.device)

logits_processor = LogitsProcessorList(
    [
        MinLengthLogitsProcessor(10, eos_token_id=model.generation_config.eos_token_id),
    ]
)
stopping_criteria = StoppingCriteriaList([MaxLengthCriteria(max_length=20)])

outputs = model.greedy_search(
    input_ids, logits_processor=logits_processor, stopping_criteria=stopping_criteria
)

tokenizer.batch_decode(outputs, skip_special_tokens=True)

['Назви найбільші міста України та коротко опиши їх. Розкажи детальніше про Львів.']

In [9]:
conversation = Conversation()
conversation = generator(conversation, pad_token_id=generator.tokenizer.eos_token_id)
print(conversation.messages[-1]["content"])

/Users/rkovalch/miniconda3/envs/simulationllm/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `4` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/Users/rkovalch/miniconda3/envs/simulationllm/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:427: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


Основні міста України, які заслуговують на згадування, це Київ, Харків, Херсон, Дніпро, Одеса, Львів, Харцизьк, Запоріжжя, Миколаїв, Кривий Ріг. Проте, якщо слідкувати за розмірами, населенням та історичною вагою, то Київ - столиця, найбільше місто, де зосереджена політика, культура, наука. Харків - інноваційний, науковий, промисловий центр, з багатою історією. Але тепер розкажуть детальніше про Львів. Львів - місто в західній Україні, відоме своєю багатовіковою історією, архітектурою, культурною спадщиною. Він є культурною столицею Західної України, з багатьма музеями, театрами, галере


In [2]:
print(torch.backends.mps.is_available())

True


In [3]:
from peft import LoraConfig, TaskType, get_peft_model

peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=False,
    r=4,
    lora_alpha=32,
    lora_dropout=0.1,
)

peft_model = get_peft_model(model, peft_config)
peft_model.print_trainable_parameters()

'NoneType' object has no attribute 'cadam32bit_grad_fp32'
trainable params: 1,703,936 || all params: 7,243,436,032 || trainable%: 0.023523863432663224


/Users/rkovalch/miniconda3/envs/simulationllm/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [8]:
peft_model()

PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralSdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=4, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=4, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear(in_features=4096, out_features=1024, bias=False)


In [4]:
from src.packages.constants.error_constants import ErrorConstants
from src.packages.prompts.instruction_tuning_gec_prompts import InstructionTuningGecPrompts
from ua_gec import Corpus
corpus = Corpus(partition="train", annotation_layer="gec-only")
for doc in corpus:
    print("\n---Source starts:---\n")
    print(doc.source)         # "I likes it."
    print("\n---Source ends:---\n")
    print("\n---Target starts:---\n")
    print(doc.target)         # "I like it."
    print("\n---Target ends---\n")
    print("\n---Annotation starts:---\n")
    print(doc.annotated)      # <AnnotatedText("I {likes=>like} it.")
    print(doc.meta.region)    # "Київська"
    print("\n---Annotation ends")
    print("\n---Prompt starts")
    prompt = InstructionTuningGecPrompts.PROMPT.format_prompt(
        query=doc.source,
        error_types=ErrorConstants.ERROR_TYPES
    )
    print(prompt)
    break


---Source starts:---

Byte for France або “Мій досвід ведення блогу у Instagram”
Останні 3 місяці мого життя видалися аж занадто насиченими на події та емоції, але ось нарешті у мене з’явилося декілька вільних годин та трохи енергії щоб продовжити серію записів щодо мого досвіду блогерства.

Сьогодні розповім про те як і навіщо мене занесло у Instagram. Якщо цікаво подивитися відразу на результат, то щиро прошу за цим посиланням – https://www.instagram.com/yevhenii_kanivets/

Моє бачення Instagram
Колись давно я прочитав статтю, чи просто коментарій – вже не згадаю. Але йшлося там про те, що найпопулярнішою соціальною мережею стане платформа на котрій можно буде лише ділитися світлинами та ставити лайки.

Було це за декілька років до появи усім відомого сервісу. Як же автору вдалося передбачити майбутнє? Дуже просто!

Instagram втілює глибинні бажання кожної людини:

емоційне збудження від перегляду гарних фото (читання – це менш природний процес, ніж споглядання)
нескінченне поглинан